<a href="https://colab.research.google.com/github/Jaredkmacho/LinearAlgebraNotebooks/blob/main/ProjectingVectors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projecting Vectors
A vector can be projected into a given other "base" vector by expresing its apparent magnitude in the same direction that said base vector.

This can be done by drawing a right angle triangle where the vector being projected acts as the hypotenuse and its components make a parallel vector and an othogonal vector to the base vector. The projected vector then will be equal to said parallel vector component.

The vector V will be composed of the sum of parallel and perpendicular vectors in relation to the base vector `V = Vpll + Vprp` where V parallel is equal to the projected vector into the base vector b `Vpll = projb(V)` 

Computing projb(V): 
- The magnitude of V parallel is equal to the Vectors dot product of the base vectors unit vector 
 `||Vpll|| = V . Ub`. 

- V parallel will point in the same direction as b, and Ub has a magnitude of 1. So this can be expresed also as the scalar product betwen the magnitude of V parallel Ub to be equal to the V parallel `||Vpll||Ub = Vpll`

- Plugging ||Vpll|| gets us `Vpll = (V . Ub)Ub`

TODO: Add a method to get the projected vector for a given base vector `Vector Vector::ProjectTo(const Vector& base)` . Also add a method to get the projected vector using the current vector as the base `Vector Vector::Project(const Vector& other)`\








In [45]:
%%writefile linear_algebra.h

#ifndef LINEAR_ALGEBRA_H
#define LINEAR_ALGEBRA_H

#include <iostream>
#include <vector>
#include <cmath>

class Vector {
public:
    Vector();
    Vector(double *coordinates, size_t len); //Constructor
    Vector(const Vector &other); //Copy constructor
    void print();
    void print(const char *name);
    double Magnitude();
    Vector Direction();
    Vector ProjectTo(Vector& base);
    Vector Project(Vector& other);
    double Dot(const Vector &other);
    double Angle(Vector &other);
    bool Parallel(Vector &other);
    bool Orthogonal(Vector &other);
    bool operator==(const Vector &other);
    Vector& operator+=(const Vector &other);
    Vector& operator-=(const Vector &other);
    Vector& operator*=(const Vector &other);
    Vector& operator*=(double x);
    

private:
    std::vector<double> coordinates_;
    size_t dimension_;
};

Vector operator+(Vector v1, const Vector &v2);
Vector operator-(Vector v1, const Vector &v2);
Vector operator*(Vector v1, const Vector &v2);
Vector operator*(Vector v, double x);


#endif /* LINEAR_ALGEBRA_H */

Overwriting linear_algebra.h


In [50]:
%%writefile linear_algebra.cpp

#include "linear_algebra.h"
Vector::Vector() {
    coordinates_ = {};
    dimension_ = 0;
}

Vector::Vector(double *coordinates, size_t len) {
    for(size_t i = 0; i < len; i++) {
        coordinates_.emplace_back(coordinates[i]);
    }
    dimension_ = len;
}

Vector::Vector(const Vector &other) {
    for(double v : other.coordinates_) {
        coordinates_.emplace_back(v);
    }
    dimension_ = other.dimension_;
}

void Vector::print() {
    std::cout << "Vector: [";
    std::cout.precision(4);
    for(size_t i = 0; i < dimension_; i++) {
        std::cout <<coordinates_[i] << ((i == dimension_ - 1) ? "]" : ",");
    }
    std::cout << std::endl;
}

void Vector::print(const char *name) {
    std::cout << name << ": [";
    std::cout.precision(4);
    for(size_t i = 0; i < dimension_; i++) {
        std::cout <<coordinates_[i] << ((i == dimension_ - 1) ? "]" : ",");
    }
    std::cout << std::endl;
}

double Vector::Magnitude() {
    double tmp = 0;
    for(double v: coordinates_) {
        tmp += std::pow(v, 2);
    }
    return std::sqrt(tmp);
}

Vector Vector::Direction() {
    Vector unit(*this);
    if(unit.Magnitude() == 0) {
        std::cout << "Zero magnitude vector!" << std::endl;
        return Vector();
    }
    double tmp = 1 / unit.Magnitude();
    return unit*=tmp;
}

//TODO: Generate a projection of this vector into a given base vector. Vpll = (V . Ub)Ub.
Vector Vector::ProjectTo(Vector& base){
    auto Ub = base.Direction();
    auto dotres = this->Dot(Ub);
    auto Vpll = Ub * dotres;
    return Vpll;
}

//TODO: Generate a projection of a given vector into this base vector. 
Vector Vector::Project(Vector& other) {
    auto Ub = this->Direction();
    auto dotres = other.Dot(Ub);
    auto Vpll = Ub * dotres;

    return Vpll;
}

double Vector::Dot(const Vector &other) {
    double tmp = 0;
    if(dimension_ == other.dimension_) {
        for(size_t i = 0; i < dimension_; i++) {
            tmp += coordinates_[i] * other.coordinates_[i];
        }
        return tmp;
    } 
    std::cout << "Not equal sized vectors!" << std::endl;
    return -1;
}

double Vector::Angle(Vector &other) {
    if(dimension_ == other.dimension_) {
        return std::acos( Dot(other) / (Magnitude() * other.Magnitude()) );
    }
    std::cout << "Not equal sized vectors!" << std::endl;
    return -1;
}

bool Vector::Parallel(Vector &other) {
    //TODO: Check weather the "other" vector is a scalar product of the "this" vector, also 0 * v1 and v1 * v1 are parallel.
    // is "other" equal to "this"
    if(*this == other) {
        return true;
    }
    // is "other" or "this" a 0 vector? 
    else if( this->Magnitude() == 0.0 || other.Magnitude() == 0.0) {
        return true;
    }
    else if(std::cos(this->Angle(other)) >= 0.99 || std::cos(this->Angle(other)) <= -0.99) {
        return true;
    } 
    return false;
}

bool Vector::Orthogonal(Vector &other) {
    //TODO: Check weather a dot product is equal to 0
    return Dot(other) <= 0.01 && Dot(other) >= -0.01;
}

bool Vector::operator==(const Vector &other) {
    return coordinates_ == other.coordinates_;
}

Vector& Vector::operator+=(const Vector &other) {
    if(dimension_ == other.dimension_) {
        for(size_t i = 0; i < dimension_; i++) {
            coordinates_[i] += other.coordinates_[i];
        }
    return *this;
    }
}

Vector operator+(Vector v1, const Vector &v2) {
    v1 += v2;
    return v1;
}

Vector& Vector::operator-=(const Vector &other) {
    if(dimension_ == other.dimension_) {
        for(size_t i = 0; i < dimension_; i++) {
            coordinates_[i] = coordinates_[i] - other.coordinates_[i];
        }
    return *this;
    }
}

Vector operator-(Vector v1, const Vector &v2) {
    v1 -= v2;
    return v1;
}

Vector& Vector::operator*=(const Vector &other) {
    if(dimension_ == other.dimension_) {
        for(size_t i = 0; i < dimension_; i++) {
            coordinates_[i] *= other.coordinates_[i];
        }
    return *this;
    }
}

Vector operator*(Vector v1, const Vector &v2) {
    v1 *= v2;
    return v1;
}

Vector& Vector::operator*=(double x) {
    for(size_t i = 0; i < dimension_; i++) {
        coordinates_[i] *= x;
    }
    return *this;
}

Vector operator*(Vector v, double x) {
    v *= x;
    return v;
}


Overwriting linear_algebra.cpp


In [51]:
%%writefile main.cpp

#include "linear_algebra.h"


int main() {
    //Example 1
    double coordinates1[] = {3.039, 1.879}; // V
    double coordinates2[] = {0.825, 2.036}; // b
    //Example 2
    double coordinates3[] = {-9.88, -3.264, -8.159};  // V
    double coordinates4[] = {-2.155, -9.353, -9.473}; // b
    //Example 3
    double coordinates5[] = {3.009, -6.172, 3.692, -2.51}; // V
    double coordinates6[] = {6.404, -9.144, 2.759, 8.718}; // b

    Vector vector1(coordinates1, sizeof(coordinates1) / sizeof(double));
    Vector vector2(coordinates2, sizeof(coordinates2) / sizeof(double));
    Vector vector3(coordinates3, sizeof(coordinates3) / sizeof(double));
    Vector vector4(coordinates4, sizeof(coordinates4) / sizeof(double));
    Vector vector5(coordinates5, sizeof(coordinates5) / sizeof(double));
    Vector vector6(coordinates6, sizeof(coordinates6) / sizeof(double));

    std::cout << "---------- Vector Prjections ---------" << std::endl;
    std::cout << "\nExample 1" << std::endl;
    vector1.print("V");
    vector2.print("b");
    auto proj1 = vector1.ProjectTo(vector2);
    proj1.print("Projb(V)");

    std::cout << "\nExample 2" << std::endl;
    vector3.print("V");
    vector4.print("b");
    auto Vper2 = vector3 - vector3.ProjectTo(vector4);
    Vper2.print("Vperp");

    std::cout << "\nExample 3" << std::endl;
    vector5.print("V");
    vector6.print("b");
    auto Vpll3 = vector5.ProjectTo(vector6);
    auto Vper3 = vector5 - Vpll3;
    Vpll3.print("Vpll");
    Vper3.print("Vper");

}

Overwriting main.cpp


In [52]:
%%script bash
g++ linear_algebra.cpp main.cpp -std=c++11 -o linal

In [53]:
!./linal

---------- Vector Prjections ---------

Example 1
V: [3.039,1.879]
b: [0.825,2.036]
Projb(V): [1.083,2.672]

Example 2
V: [-9.88,-3.264,-8.159]
b: [-2.155,-9.353,-9.473]
Vperp: [-8.35,3.376,-1.434]

Example 3
V: [3.009,-6.172,3.692,-2.51]
b: [6.404,-9.144,2.759,8.718]
Vpll: [1.969,-2.811,0.8481,2.68]
Vper: [1.04,-3.361,2.844,-5.19]
